In [1]:
import os
import sys
import torch
import pandas as pd
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoModelForImageClassification,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM
)


# --- Path and Module Setup ---
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path: sys.path.append(src_path)

from pipeline import compress_model, decompress_model
from utils import calculate_delta_parameters, calculate_parameters_size, calculate_compressed_size
from evaluation import evaluate_accuracy
from runner import run_classification_experiment


# Configure pandas for better table display
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 140)

print("Setup complete.")


Setup complete.


## Basic Testing

In [ ]:
# Load Models
pretrained_model = AutoModelForSequenceClassification.from_pretrained("roberta-base")
finetuned_model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-SST-2")

# Load and Prepare Dataset
eval_dataset = load_dataset("glue", "sst2", split="validation")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
def tokenize_function(e): return tokenizer(e["sentence"], padding="max_length", truncation=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataloader = DataLoader(tokenized_eval_dataset, batch_size=16)

print("Models and data loaded.")

In [ ]:
# --- Compression ---
compressed_data = compress_model(
    pretrained_model,
    finetuned_model,
    patch_size=8,
    bit_strategy=[(2, 0.5), (0, 0.5)]
)

# --- Decompression ---
reconstructed_model = decompress_model(pretrained_model, compressed_data)
print("\nCompression and Decompression pipelines finished.")

In [ ]:
# --- Accuracy Evaluation ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

original_accuracy = evaluate_model_accuracy(finetuned_model, eval_dataloader, device)
reconstructed_accuracy = evaluate_model_accuracy(reconstructed_model, eval_dataloader, device)

print("\n--- Accuracy Comparison ---")
print(f"Original fine-tuned model accuracy: {original_accuracy:.4f}")
print(f"Reconstructed model accuracy:   {reconstructed_accuracy:.4f}")
print(f"Accuracy drop: {(original_accuracy - reconstructed_accuracy):.4f}")

# --- Storage Size Evaluation ---
uncompressed_delta_weights = calculate_delta_parameters(pretrained_model, finetuned_model)
original_delta_size = calculate_parameters_size(uncompressed_delta_weights)
compressed_delta_size = calculate_compressed_size(compressed_data)

print("\n--- Storage Size Comparison ---")
print(f"Original delta parameters size:  {original_delta_size:.2f} MB")
print(f"Compressed delta data size:      {compressed_delta_size:.2f} MB")
if compressed_delta_size > 0:
    print(f"Compression Ratio: {(original_delta_size / compressed_delta_size):.2f}x")

In [ ]:
all_results = []
device_to_use = "cpu"

# --- Experiment Definitions ---
pretrained_id = "roberta-base"
finetuned_variants = [
    "textattack/roberta-base-SST-2",
    # You can add other RoBERTa models fine-tuned on SST-2 here if you find them
]
patch_sizes = [8, 16, 32]
bit_strategies = [
    [(2, 0.5), (0, 0.5)],
    [(4, 0.5), (0, 0.5)]
]

# --- Main Loop ---
for finetuned_id in finetuned_variants:
    for p_size in patch_sizes:
        for bit_strat in bit_strategies:
            result = run_classification_experiment(
                pretrained_model_id=pretrained_id,
                finetuned_model_id=finetuned_id,
                patch_size=p_size,
                bit_strategy=bit_strat,
                device=device_to_use
            )
            all_results.append(result)

In [ ]:
# In model_evaluation.ipynb (Cell 2)

# This list defines the groups of experiments we want to run.
experiments_config_groups = [
    {
        "group_name": "RoBERTa-SST2",
        "pretrained_model_id": "roberta-base",
        "finetuned_model_id": "textattack/roberta-base-SST-2",
        "model_class": AutoModelForSequenceClassification,
        "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
        # --- Define lists of hyperparameters to test ---
        "patch_sizes": [8, 16, 32],
        "bit_strategies": [
            [(2, 0.5), (0, 0.5)],
            [(4, 0.5), (0, 0.5)]
        ]
    },
    {
        "group_name": "ViT-CIFAR10",
        # --- FIX IS HERE: Corrected the typo from 'in2k' to 'in21k' ---
        "pretrained_model_id": "google/vit-base-patch16-224-in21k",
        "finetuned_model_id": "nateraw/vit-base-patch16-224-cifar10",
        "model_class": AutoModelForImageClassification,
        "task_info": {"name": "cifar10", "config": None, "split": "test"},
        "patch_sizes": [16],
        "bit_strategies": [
            [(2, 0.5), (0, 0.5)]
        ]
    }
]

In [ ]:
# In model_evaluation.ipynb (Cell 3)

all_results = []
device_to_use = "cpu"

# Loop through each group of experiments
for experiment_group in experiments_config_groups:
    # Loop through each hyperparameter combination
    for p_size in experiment_group["patch_sizes"]:
        for bit_strat in experiment_group["bit_strategies"]:
            
            # --- Create the final, flat config for this specific run ---
            config = {
                "pretrained_model_id": experiment_group["pretrained_model_id"],
                "finetuned_model_id": experiment_group["finetuned_model_id"],
                "model_class": experiment_group["model_class"],
                "task_info": experiment_group["task_info"],
                "patch_size": p_size,      # Use the singular key 'patch_size'
                "bit_strategy": bit_strat  # Use the singular key 'bit_strategy'
            }

            try:
                result = run_classification_experiment(
                    config=config,
                    device=device_to_use
                )
                all_results.append(result)
            except Exception as e:
                print(f"\n!!!!!! An error occurred during experiment: {config.get('finetuned_model_id')} !!!!!!")
                print(f"Error: {e}\n")

print("\n\nAll experiments have been completed.")

In [ ]:
if all_results:
    results_df = pd.DataFrame(all_results)
    print("\n--- All Experiment Results ---")
    display(results_df)
else:
    print("No results to display. Please check for errors in the previous cell.")

## Multimodal Testing

In [ ]:
experiments_config = [
    # {
    #     "pretrained_model_id": "roberta-base",
    #     "finetuned_model_id": "textattack/roberta-base-SST-2",
    #     "model_class": AutoModelForSequenceClassification,
    #     "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
    #     "patch_size": 16,
    #     "bit_strategy": [(2, 0.5), (0, 0.5)]
    # },
    {
        "pretrained_model_id": "google/vit-base-patch16-224-in21k",
        "finetuned_model_id": "nateraw/vit-base-patch16-224-cifar10",
        "model_class": AutoModelForImageClassification,
        "task_info": {"name": "cifar10", "config": None, "split": "test"},
        "patch_size": 16,
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    },
    # {
    # "pretrained_model_id": "distilbert-base-uncased",
    # "finetuned_model_id": "distilbert-base-uncased-finetuned-sst-2-english",
    # "model_class": AutoModelForSequenceClassification,
    # "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
    # "patch_size": 16,
    # "bit_strategy": [(2, 0.5), (0, 0.5)]
    # },
        # --- NEW: Swin Transformer (Image Classification) Experiment ---
    # {
    #     "pretrained_model_id": "microsoft/swin-tiny-patch4-window7-224",
    #     "finetuned_model_id":  "rs127/swin-tiny-patch4-window7-224-finetuned-cifar10",
    #     "model_class": AutoModelForImageClassification,
    #     "task_info": {"name": "cifar10", "config": None, "split": "test"},
    #     "patch_size": 16, # You can experiment with other patch sizes like 8 or 32
    #     "bit_strategy": [(2, 0.5), (0, 0.5)]
    # }

]

In [ ]:
all_results = []
device_to_use = "cpu"

for config in experiments_config:
    try:
        result = run_classification_experiment(
            config=config,
            device=device_to_use
        )
        all_results.append(result)
    except Exception as e:
        print(f"\n!!!!!! An error occurred during experiment: {config.get('finetuned_model_id')} !!!!!!")
        print(f"Error: {e}\n")

print("\n\nAll experiments have been completed.")

In [ ]:
if all_results:
    results_df = pd.DataFrame(all_results)
    print("\n--- Experiment Results ---")
    display(results_df)
else:
    print("No results to display. Please check for errors in the previous cell.")

In [ ]:
from huggingface_hub import list_models

def find_huggingface_models(search_query: str, limit: int = 10):
    """
    Searches the Hugging Face Hub for models matching a query and prints a sorted list.

    Args:
        search_query (str): The term to search for (e.g., "swin tiny cifar10").
        limit (int): The maximum number of results to display.
    """
    print(f"--- Searching for models matching: '{search_query}' ---")
    
    # list_models returns a generator of models. We sort them by download count.
    models = list(list_models(
        search=search_query,
        sort="downloads",
        direction=-1,
        limit=limit
    ))
    
    if not models:
        print("No models found.")
        return

    print(f"Found {len(models)} models (sorted by popularity):\n")
    for model in models:
        print(f"ID: {model.modelId}")
        print(f"  Task: {model.pipeline_tag} | Downloads: {model.downloads}\n")

# --- Run a search to find a Swin Transformer ---
# This will find popular Swin Tiny models fine-tuned on CIFAR-10
find_huggingface_models("swin tiny cifar10")

## DWT

In [ ]:
experiments_config = [
    # --- RoBERTa Experiment with DCT ---
    {
        "transform_type": "dct",
        "pretrained_model_id": "roberta-base",
        "finetuned_model_id": "textattack/roberta-base-SST-2",
        "model_class": AutoModelForSequenceClassification,
        "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
        "patch_size": 16,
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    },
    # --- RoBERTa Experiment with DWT ---
    {
        "transform_type": "dwt",
        "pretrained_model_id": "roberta-base",
        "finetuned_model_id": "textattack/roberta-base-SST-2",
        "model_class": AutoModelForSequenceClassification,
        "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
        "patch_size": 16, # Patch sizes that are powers of 2 are good for DWT
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    },

    # --- ViT Experiment with DCT ---
    {
        "transform_type": "dct",
        "pretrained_model_id": "google/vit-base-patch16-224-in21k",
        "finetuned_model_id": "nateraw/vit-base-patch16-224-cifar10",
        "model_class": AutoModelForImageClassification,
        "task_info": {"name": "cifar10", "config": None, "split": "test"},
        "patch_size": 16,
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    },
    # --- ViT Experiment with DWT ---
    {
        "transform_type": "dwt",
        "pretrained_model_id": "google/vit-base-patch16-224-in21k",
        "finetuned_model_id": "nateraw/vit-base-patch16-224-cifar10",
        "model_class": AutoModelForImageClassification,
        "task_info": {"name": "cifar10", "config": None, "split": "test"},
        "patch_size": 16,
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    }
]

In [ ]:
all_results = []
device_to_use = "cpu"

for config in experiments_config:
    try:
        result = run_classification_experiment(
            config=config,
            device=device_to_use
        )
        all_results.append(result)
    except Exception as e:
        print(f"\n!!!!!! An error occurred during experiment: {config.get('finetuned_model_id')} !!!!!!")
        print(f"Transform: {config.get('transform_type')}")
        print(f"Error: {e}\n")

print("\n\nAll experiments have been completed.")

In [ ]:
if all_results:
    results_df = pd.DataFrame(all_results)
    print("\n--- All Experiment Results ---")
    display(results_df)
else:
    print("No results to display. Please check for errors in the previous cell.")

## Table Quantization

In [14]:
experiments_config = [
    # --- RoBERTa Experiment 1: DCT without JPEG Quantization (Baseline) ---
    {
        "transform_type": "dct",
        "use_jpeg_quantization": False, # JPEG feature is OFF
        "pretrained_model_id": "roberta-base",
        "finetuned_model_id": "textattack/roberta-base-SST-2",
        "model_class": AutoModelForSequenceClassification,
        "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
        "patch_size": 64, # Using 16x16 patch for a direct comparison
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    },
    
    # --- RoBERTa Experiment 2: DCT with JPEG Quantization ---
    {
        "transform_type": "dct",
        "use_jpeg_quantization": True,  # JPEG feature is ON
        "pretrained_model_id": "roberta-base",
        "finetuned_model_id": "textattack/roberta-base-SST-2",
        "model_class": AutoModelForSequenceClassification,
        "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
        "patch_size": 64,
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    },


    # --- ViT Experiment with DCT ---
    {
        "transform_type": "dct",
        "use_jpeg_quantization": False, # JPEG feature is OFF
        "pretrained_model_id": "google/vit-base-patch16-224-in21k",
        "finetuned_model_id": "nateraw/vit-base-patch16-224-cifar10",
        "model_class": AutoModelForImageClassification,
        "task_info": {"name": "cifar10", "config": None, "split": "test"},
        "patch_size": 64,
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    },
    # --- ViT Experiment with DWT ---
    {
        "transform_type": "dct",
        "use_jpeg_quantization": True,  # JPEG feature is ON
        "pretrained_model_id": "google/vit-base-patch16-224-in21k",
        "finetuned_model_id": "nateraw/vit-base-patch16-224-cifar10",
        "model_class": AutoModelForImageClassification,
        "task_info": {"name": "cifar10", "config": None, "split": "test"},
        "patch_size": 64,
        "bit_strategy": [(2, 0.5), (0, 0.5)]
    }

    # # --- RoBERTa Experiment 3: DWT without JPEG Quantization ---
    # {
    #     "transform_type": "dwt",
    #     "use_jpeg_quantization": False, # JPEG feature is OFF
    #     "pretrained_model_id": "roberta-base",
    #     "finetuned_model_id": "textattack/roberta-base-SST-2",
    #     "model_class": AutoModelForSequenceClassification,
    #     "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
    #     "patch_size": 32,  # Using 16x16 patch for a direct comparison
    #     "bit_strategy": [(2, 0.5), (0, 0.5)]
    # },
    
    # # --- RoBERTa Experiment 4: DWT with JPEG Quantization ---
    # {
    #     "transform_type": "dwt",
    #     "use_jpeg_quantization": True,  # JPEG feature is ON
    #     "pretrained_model_id": "roberta-base",
    #     "finetuned_model_id": "textattack/roberta-base-SST-2",
    #     "model_class": AutoModelForSequenceClassification,
    #     "task_info": {"name": "glue", "config": "sst2", "split": "validation", "text_column": "sentence"},
    #     "patch_size": 32,
    #     "bit_strategy": [(2, 0.5), (0, 0.5)]
    # }
]

In [15]:
all_results = []
device_to_use = "cpu"

for config in experiments_config:
    try:
        # The runner will now check for the 'use_jpeg_quantization' key
        result = run_classification_experiment(
            config=config,
            device=device_to_use
        )
        all_results.append(result)
    except Exception as e:
        print(f"\n!!!!!! An error occurred during experiment: {config.get('finetuned_model_id')} !!!!!!")
        print(f"Transform: {config.get('transform_type')}, JPEG: {config.get('use_jpeg_quantization')}")
        print(f"Error: {e}\n")

print("\n\nAll experiments have been completed.")

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



==================== Starting Experiment: roberta-base-SST-2 ====================
Loading models: roberta-base and textattack/roberta-base-SST-2


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preparing dataloader for dataset: glue
Starting model compression using transform: DCT...
Compressing layer: roberta.embeddings.word_embeddings.weight...
Compressing layer: roberta.embeddings.position_embeddings.weight...
Skipping compression for layer 'roberta.embeddings.token_type_embeddings.weight'. Storing uncompressed.
Skipping compression for layer 'roberta.embeddings.LayerNorm.weight'. Storing uncompressed.
Skipping compression for layer 'roberta.embeddings.LayerNorm.bias'. Storing uncompressed.
Compressing layer: roberta.encoder.layer.0.attention.self.query.weight...
Skipping compression for layer 'roberta.encoder.layer.0.attention.self.query.bias'. Storing uncompressed.
Compressing layer: roberta.encoder.layer.0.attention.self.key.weight...
Skipping compression for layer 'roberta.encoder.layer.0.attention.self.key.bias'. Storing uncompressed.
Compressing layer: roberta.encoder.layer.0.attention.self.value.weight...
Skipping compression for layer 'roberta.encoder.layer.0.attent

Evaluating Accuracy:   0%|          | 0/7 [00:00<?, ?it/s]/Users/mrsadeghian/Desktop/delta_dct_project/delta_dct_env/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Evaluating Accuracy: 100%|██████████| 7/7 [00:04<00:00,  1.40it/s]


...

Releasing models from memory...
==================== Experiment Finished ====================

==================== Starting Experiment: vit-base-patch16-224-cifar10 ====================
Loading models: google/vit-base-patch16-224-in21k and nateraw/vit-base-patch16-224-cifar10


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preparing dataloader for dataset: cifar10
JPEG-style quantization ENABLED.
Starting model compression using transform: DCT...
Skipping compression for layer 'vit.embeddings.cls_token'. Storing uncompressed.
Skipping compression for layer 'vit.embeddings.position_embeddings'. Storing uncompressed.
Skipping compression for layer 'vit.embeddings.patch_embeddings.projection.weight'. Storing uncompressed.
Skipping compression for layer 'vit.embeddings.patch_embeddings.projection.bias'. Storing uncompressed.
Compressing layer: vit.encoder.layer.0.attention.attention.query.weight...
Skipping compression for layer 'vit.encoder.layer.0.attention.attention.query.bias'. Storing uncompressed.
Compressing layer: vit.encoder.layer.0.attention.attention.key.weight...
Skipping compression for layer 'vit.encoder.layer.0.attention.attention.key.bias'. Storing uncompressed.
Compressing layer: vit.encoder.layer.0.attention.attention.value.weight...
Skipping compression for layer 'vit.encoder.layer.0.atten

Evaluating Accuracy: 100%|██████████| 7/7 [00:08<00:00,  1.19s/it]

Releasing models from memory...
==================== Experiment Finished ====================


All experiments have been completed.


In [4]:
if all_results:
    results_df = pd.DataFrame(all_results)
    print("\n--- All Experiment Results ---")
    display(results_df)
else:
    print("No results to display. Please check for errors in the previous cell.")


--- All Experiment Results ---


,model_name,transform,jpeg_quant,patch_size,bit_strategy,original_accuracy,reconstructed_accuracy,accuracy_drop,original_delta_mb,compressed_delta_mb,compression_ratio
0,roberta-base-SST-2,dct,False,64,"[(2, 0.5), (0, 0.5)]",0.945,0.940,0.005,475.491219,121.336723,3.918774
1,roberta-base-SST-2,dct,True,64,"[(2, 0.5), (0, 0.5)]",0.945,0.945,0.000,475.491219,121.336723,3.918774
2,roberta-base-SST-2,dwt,False,64,"[(2, 0.5), (0, 0.5)]",0.945,0.930,0.015,475.491219,122.029686,3.896521
3,roberta-base-SST-2,dwt,True,64,"[(2, 0.5), (0, 0.5)]",0.945,0.625,0.320,475.491219,122.029686,3.896521


In [16]:
if all_results:
    results_df = pd.DataFrame(all_results)
    print("\n--- All Experiment Results ---")
    display(results_df)
else:
    print("No results to display. Please check for errors in the previous cell.")


--- All Experiment Results ---


,model_name,transform,jpeg_quant,patch_size,bit_strategy,original_accuracy,reconstructed_accuracy,accuracy_drop,original_delta_mb,compressed_delta_mb,compression_ratio
0,roberta-base-SST-2,dct,False,64,"[(2, 0.5), (0, 0.5)]",0.945,0.940,0.005,475.491219,121.336723,3.918774
1,roberta-base-SST-2,dct,True,64,"[(2, 0.5), (0, 0.5)]",0.945,0.945,0.000,475.491219,121.336723,3.918774
2,vit-base-patch16-224-cifar10,dct,False,64,"[(2, 0.5), (0, 0.5)]",0.995,0.980,0.015,327.325233,84.562538,3.870807
3,vit-base-patch16-224-cifar10,dct,True,64,"[(2, 0.5), (0, 0.5)]",0.995,0.980,0.015,327.325233,84.562538,3.870807
